# Rename core photos

---

<img src="img/renamecore.jpg" width="800" />

---

## How to use

1. Run the cell below (click it and hit Shiht-Enter)
2. Click `Select` and navigate to the photos directory

    Hint: click `...` to move up a directory. When a name is displayed in the top dropbox click on it to change drive.

3. Once at the correct directory, click `Change`
3. The first photo is now displayed with its name
3. Fill-in the info for that photo and click `Rename photo`

    The photo is renamed straight away and the next one is shown

4. Repeat step 3...

Renaming is trying to get smart:

- Only Dry/Wet state will change between two consecutive photos if `2 photos per interval` is checked
- `Box number` increment by one from one photo to the next
- `mFrom` is the previous `mTo` from one photo to the next

To skip a photo and go to the next one, click: `Skip photo`.

You can stop at any time, just close the Notebook.

You can also restart at any time by running (Shiht-Enter) the cell again

## Run hidden cell below:

In [ ]:
import os
from ipyfilechooser import FileChooser
import ipywidgets as widgets

photolst = []
nphoto = 0
dw = 0
spcs = '&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; |&nbsp;&nbsp;&nbsp;&nbsp;'


# ----------------------------------------------------------------
def f(v):
    """Format number."""
    #
    return '%0.2f' % v


def nextphoto(inc=True):
    """Display next photo."""
    #
    global nphoto
    #
    if inc:
        nphoto += 1
    if nphoto >= len(photolst):
        nphoto = 0
        labl.value = "<b><font color='red'>WARNING: This is the last photo!</font></b>"
        shownp(0)
        file = open(photolst[-1], "rb")
        return file.read()
    else:
        labl.value = "<font color='black'>%s</font>" % photolst[nphoto]
        file = open(photolst[nphoto], "rb")
        return file.read()


def donext():
    """Change display for next photo."""
    #
    mfrom.value = mto.value
    boxnb.value += 1


def saveren(L):
    """Save current rename operation."""
    #
    with open(os.path.join(fc.selected_path, 'Rename_Photos_List.csv'), 'a') as fo:
        fo.write('%s\n' % (os.path.split(photolst[nphoto])[1] + ',' + L))


def shownp(n):
    """Show renaming progress."""
    #
    labn.value = spcs + 'Number of photos to rename: ' + str(n) + '/' + str(len(photolst))


# Callbacks
def rename_photo(btn):
    """Rename the current photo and jump to next one."""
    #
    global nphoto, dw
    #
    # Rename
    new_name = str(holeid.value).strip() + '_' + f(mfrom.value) + '-' + f(mto.value) + 'm_Box' + str(boxnb.value) + '_'
    new_name += 'D' if drywet.value == 1 else 'W'
    new_name += os.path.splitext(photolst[nphoto])[1]

    #print(new_name, photolst[nphoto])
    try:
        os.rename(photolst[nphoto], os.path.join(fc.selected_path, new_name))
        saveren(new_name)
    except:
        labl.value = "<b><font color='red'>ERROR: cannot rename photo!</font></b>"
        saveren('ERROR: cannot rename photo!')
    #
    # Setup new info
    if dw == 0 and twoimage.value:
        drywet.value = 2 if drywet.value == 1 else 1
        dw = 1
    elif twoimage:
        drywet.value = 2 if drywet.value == 1 else 1
        donext()
        dw = 0
    else:
        donext()
    #
    photo.value = nextphoto()
    shownp(len(photolst) - nphoto)


def to_next(btn):
    """Skip to next photo."""
    #
    global nphoto
    #
    photo.value = nextphoto()


def changed_dir(chooser):
    """Choose the folder with photos to rename."""
    #
    global photolst, nphoto
    #
    pth_photos = chooser.selected_path
    photolst = [os.path.join(pth_photos, f) for f in os.listdir(pth_photos) if os.path.splitext(f)[1].lower() in ['.jpg', '.png']]
    nphoto = 0
    photo.value = nextphoto(False)
    shownp(len(photolst))
    with open(os.path.join(pth_photos, 'Rename_Photos_List.csv'), 'w') as fo:
        fo.write('Old_Name,New_Name\n')


# ----------------------------------------------------------------
# Define widgets
labl = widgets.HTML(value=' ')
labn = widgets.HTML(value=' ')
twoimage = widgets.Checkbox(
    value=True,
    description='2 photos per interval',
    disabled=False,
    indent=False
)
coretype = widgets.Dropdown(
    options=[('Whole core', 1), ('Half core', 2), ('Quarter Core', 3)],
    value=1,
    description='Core Type:',
)
drywet = widgets.Dropdown(
    options=[('Dry', 1), ('Wet', 2)],
    value=1,
    description='Dry or Wet:',
)
holeid = widgets.Text(
    value='',
    placeholder='Hole ID',
    description='Hole ID:',
    disabled=False
)
boxnb = widgets.BoundedIntText(
    value=1,
    min=1,
    max=9000,
    step=1,
    description='Box number:',
    disabled=False
)
mfrom = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=10000,
    step=3,
    description='mFrom:',
    disabled=False
)
mto = widgets.BoundedFloatText(
    value=3,
    min=0,
    max=10000,
    step=3,
    description='mTo:',
    disabled=False
)
rena = widgets.Button(
    description='Rename photo',
    disabled=False,
    button_style='',
    tooltip='Click to rename photo',
    icon='camera-rotate'
)
skp = widgets.Button(
    description='Skip photo',
    disabled=False,
    button_style='',       # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to skip this photo',
    icon='angles-right'    # (FontAwesome names without the `fa-` prefix)
)
photo = widgets.Image(
    value=b'',
    format='jpg',
    width=900,
    height=600,
)
fc = FileChooser('')

# Setup callbacks
rena.on_click(rename_photo)
skp.on_click(to_next)
fc.register_callback(changed_dir)

# Setup UI
#items_layout = widgets.Layout(width='auto')     # override the default width of the button to 'auto' to let the button grow
box_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    border=None,
    width='100%'
)
col_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    width='900px',
    border='solid 1px',
    padding='3px'
)
btn_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    border='solid 1px',
    padding='3px',
    margin='auto'
)
lbl_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    width='900px',
    align_items='stretch'
)

labls = widgets.Box(children=[labl, labn], layout=lbl_layout)
btns = widgets.Box(children=[rena, skp], layout=btn_layout)
litems = [twoimage, holeid, mfrom, mto]
ritems = [btns, boxnb, coretype, drywet]
lbox = widgets.Box(children=litems, layout=box_layout)
rbox = widgets.Box(children=ritems, layout=box_layout)
bbox = widgets.Box(children=[lbox, rbox], layout=col_layout)

box = widgets.Box(children=[fc, labls, bbox, photo], layout=box_layout)
box